In [94]:
import pandas as pd
import numpy as np
import operator

In [95]:
#-----------------
# read input csv
#-----------------
trace_file = "nvprof_gputrace.csv"

# There are max 17 columns in the output csv
max_cols = ["Start","Duration","Grid X","Grid Y","Grid Z","Block X","Block Y","Block Z","Registers Per Thread","Static SMem","Dynamic SMem","Size","Throughput","Device","Context","Stream","Name"]

df_trace = pd.read_csv(trace_file, names=max_cols, engine='python')

rows_to_skip = 0
## find out the number of rows to skip
for index, row in df_trace.iterrows():
    if row['Start'] == 'Start':
        rows_to_skip = index
        break
        
# read the input csv again 
df_trace = pd.read_csv(trace_file, skiprows=rows_to_skip)


In [96]:
df_trace

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,Device,Context,Stream,Name
0,ms,ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,B,MB,GB/s,NaN,NaN,NaN,NaN
1,218.148062,0.668229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.845676,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
2,218.817507,0.669669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.833106,GeForce GTX 950 (0),1.0,13.0,[CUDA memcpy HtoD]
3,219.488392,0.662886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.892793,GeForce GTX 950 (0),1.0,17.0,[CUDA memcpy HtoD]
4,220.152462,1.719213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,2.272115,GeForce GTX 950 (0),1.0,17.0,[CUDA memcpy HtoD]
5,220.256110,0.666438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.861385,GeForce GTX 950 (0),1.0,16.0,[CUDA memcpy HtoD]
6,220.923764,3.342458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.168676,GeForce GTX 950 (0),1.0,16.0,[CUDA memcpy HtoD]
7,221.872923,0.669189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.837290,GeForce GTX 950 (0),1.0,14.0,[CUDA memcpy HtoD]
8,222.543296,2.010256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.943160,GeForce GTX 950 (0),1.0,14.0,[CUDA memcpy HtoD]
9,222.927715,0.669701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,5.832827,GeForce GTX 950 (0),1.0,15.0,[CUDA memcpy HtoD]


In [91]:
type(df_trace)

pandas.core.frame.DataFrame

In [97]:
rows, cols = df_trace.shape
# print rows
# print cols

start_unit = df_trace['Start'].iloc[0]
# print start_unit

duration_unit = df_trace['Duration'].iloc[0]
# print duration_unit


start_coef =  1.0

if start_unit == 's':
    start_coef = 1e3
    
if start_unit == 'us':
    start_coef = 1e-3
    
# print start_coef


duration_coef =  1.0

if duration_unit == 's':
    duration_coef = 1e3
    
if duration_unit == 'us':
    duration_coef = 1e-3
    
# print duration_coef

# read the row 1 for the starting time
start_time = df_trace['Start'].iloc[1]
# print type(start_time)
start_time = float(start_time) * start_coef
# print start_time


# read the last row  for the starting time
end_time = df_trace['Start'].iloc[rows-1]
end_dur_time = df_trace['Duration'].iloc[rows-1]
# print type(end_time)
# print type(end_dur_time)
# print end_time
# print end_dur_time
end_time = float(end_time) * start_coef
end_dur_time = float(end_dur_time) * duration_coef

end_time = end_time + end_dur_time

# print end_time

#---------------------------------
# total runtime
#---------------------------------
runtime = end_time - start_time
print "runtime : " + str(runtime) + " ms"

runtime : 8.021087 ms


### stream starting time

In [99]:
# remove the 1st row
df = df_trace.drop(df_trace.index[[0]])
#df

streams_ls = list(df_trace['Stream'].unique())
# remove nan
streams_ls = [x for x in streams_ls if str(x) != 'nan']
print streams_ls

# init the dict with large values
streams_start = dict()
for x in streams_ls:
    streams_start[str(x)] = 999999999999999.0

for index, row in df.iterrows():
    print row['Start']
    print row['Stream']
    start_ms = start_coef * float(row['Start'])
    sid = row['Stream']
    
    if streams_start[str(sid)] > start_ms:
        streams_start[str(sid)] = start_ms
    
# streams_start time in ms
print streams_start

[13.0, 17.0, 16.0, 14.0, 15.0, 18.0]
218.148062
13.0
218.817507
13.0
219.488392
17.0
220.152462
17.0
220.256110
16.0
220.923764
16.0
221.872923
14.0
222.543296
14.0
222.927715
15.0
223.598632
15.0
224.554768
18.0
225.217877
18.0
{'15.0': 222.927715, '14.0': 221.872923, '17.0': 219.488392, '16.0': 220.25611, '18.0': 224.554768, '13.0': 218.148062}


In [100]:
# sort the dict in asc order
sorted_streams_start = sorted(streams_start.items(), key=operator.itemgetter(1))

print sorted_streams_start

[('13.0', 218.148062), ('17.0', 219.488392), ('16.0', 220.25611), ('14.0', 221.872923), ('15.0', 222.927715), ('18.0', 224.554768)]


In [ ]:
print type(sorted_streams_start)

In [101]:
from collections import OrderedDict
od = OrderedDict(sorted_streams_start)
print od
print type(od)

OrderedDict([('13.0', 218.148062), ('17.0', 219.488392), ('16.0', 220.25611), ('14.0', 221.872923), ('15.0', 222.927715), ('18.0', 224.554768)])
<class 'collections.OrderedDict'>


In [107]:
# min_key = min(od.iteritems(), key=operator.itemgetter(1))[0]
# print min_key
# print od[min_key]

# max_key = max(od.iteritems(), key=operator.itemgetter(1))[0]
# print max_key
# print od[max_key]

13.0
218.148062
18.0
224.554768


In [110]:
prev_stream = -1
curr_stream = -1

prev_start_ms = -1
curr_start_ms = -1

streams_lnch_ovhd = []

for key, value in od.iteritems():
    if prev_stream == -1:
        prev_stream = key
        prev_start_ms = value
        print str(prev_stream) + ' 0 (ms)'
        
    curr_stream = key
    curr_start_ms = value
    
    if curr_stream != prev_stream:
        ovhd = curr_start_ms - prev_start_ms
        print str(curr_stream) + ' ' + str(ovhd) + ' (ms)'
        
        streams_lnch_ovhd.append(ovhd)
        
        #update prev
        prev_stream = curr_stream
        prev_start_ms = curr_start_ms
        

13.0 0 (ms)
17.0 1.34033 (ms)
16.0 0.767718 (ms)
14.0 1.616813 (ms)
15.0 1.054792 (ms)
18.0 1.627053 (ms)


In [115]:
print streams_lnch_ovhd
print min(streams_lnch_ovhd)
print max(streams_lnch_ovhd)
print sum(streams_lnch_ovhd) / float(len(streams_lnch_ovhd))

[1.3403299999999945, 0.7677180000000021, 1.6168129999999792, 1.0547920000000204, 1.6270529999999894]
0.767718
1.627053
1.2813412


In [ ]:
df_trace

In [ ]:
streams = list(df_trace['Stream'].unique())

# remove nan
streams = [x for x in streams if str(x) != 'nan']
print streams

In [ ]:
# # go through each stream, find when the streams start
# for sid in streams: